<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 71 al 75 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos  modificada para ser compatible en esta ocasion, para **Oracle**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio.

#### Celda para configurar el entorno

In [ ]:
# !python -m pip install oracledb
# !pip install pandas
# !pip install sqlalchemy

#### Importamos librerias y se realiza la conexion utilizando un archivo json con la configuracion local de la base de datos

Si se desea replicar lo propio usando el script y este cuaderno, basta con crear el archivo json con el par de valores que contengan su informacion local y cargarlo al directorio.

host:

user:

password:

sid:

In [1]:
# importar las librerias
import json
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd

# Cargar los detalles de la conexión desde un archivo de configuración propio
with open('config3.json') as f:
    config = json.load(f)

# Crear la cadena de conexión
connection_str = f"oracle+oracledb://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['sid']}"

# Crear objeto engine de sqlalchemy que gestiona la conexion
engine = create_engine(connection_str)

#### Realizamos una prueba y se Inspecciona la base de datos

In [2]:
# Definir consulta en SQL compatible con Oracle
try:
    QUERY = """
    SELECT table_name FROM user_tables
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
    df = pd.read_sql_query(QUERY, engine)
# Mostrar los primeros resultados
    print("La conexión se ha realizado con éxito.\n")
    print(df)
except Exception as e:
    print("Hubo un error al realizar la conexión:", e)

La conexión se ha realizado con éxito.

     table_name
0      HOSPITAL
1          SALA
2     PLANTILLA
3     OCUPACION
4        DOCTOR
5       ENFERMO
6      EMPLEADO
7  DEPARTAMENTO


#### Inspeccion a las tablas de la base de datos 
#### Con la finalidad de observar su estructura y tipo de datos

In [3]:
for valor in df['table_name']:
    QUERY = f"SELECT column_name, data_type, data_length FROM USER_TAB_COLUMNS WHERE table_name = '{valor}'" 
    df1 = pd.read_sql_query(QUERY, engine)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla HOSPITAL

    column_name data_type  data_length
0  HOSPITAL_COD    NUMBER           22
1        NOMBRE  VARCHAR2           10
2     DIRECCION  VARCHAR2           20
3      TELEFONO  VARCHAR2            8
4      NUM_CAMA    NUMBER           22

El contenido de la tabla SALA

    column_name data_type  data_length
0  HOSPITAL_COD    NUMBER           22
1      SALA_COD    NUMBER           22
2        NOMBRE  VARCHAR2           20
3      NUM_CAMA    NUMBER           22

El contenido de la tabla PLANTILLA

    column_name data_type  data_length
0  HOSPITAL_COD    NUMBER           22
1      SALA_COD    NUMBER           22
2   EMPLEADO_NO    NUMBER           22
3      APELLIDO  VARCHAR2           15
4       FUNCION  VARCHAR2           10
5         TURNO  VARCHAR2            1
6       SALARIO    NUMBER           22

El contenido de la tabla OCUPACION

    column_name data_type  data_length
0   INSCRIPCION    NUMBER           22
1  HOSPITAL_COD    NUMBER           22
2

### Resolucion de los ejercicios numerados del 71 al 75

##### CREACIÓN DE VISTAS.



##### 71.  
- Crear una vista desde los registros de la Tabla EMPLEADOS para los departamentos 10 y 20
- Crear una vista para los departamentos 10 y 30.
- Hacer una JOIN de las dos vistas anteriores.

In [4]:
# Crear una vista desde los registros de la Tabla EMPLEADOS para los departamentos 10 y 20
with engine.begin() as connection:
    QUERY = """
    CREATE VIEW DEP_10_20 AS
    SELECT *
    FROM EMPLEADO
    WHERE DEPT_NO IN (10,20)
    """
    result = connection.execute(text(QUERY))

In [5]:
# verificar la creacion de la vista por el nombre de la vista
QUERY = """
    SELECT owner, view_name, text
    FROM ALL_VIEWS
    WHERE VIEW_NAME = 'DEP_10_20'
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
pd.set_option('display.max_colwidth', None)
df

,owner,view_name,text
0,USUARIO_OSCAR,DEP_10_20,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,20)"


In [6]:
# Crear una vista desde los registros de la Tabla EMPLEADOS para los departamentos 10 y 30
with engine.begin() as connection:
    QUERY = """
    CREATE VIEW DEP_10_30 AS
    SELECT *
    FROM EMPLEADO
    WHERE DEPT_NO IN (10,30)
    """
    result = connection.execute(text(QUERY))

In [7]:
# verificar la creacion de la vista recuperando todas las creadas en el esquema
QUERY = """
    SELECT owner, view_name, text
    FROM ALL_VIEWS
    WHERE owner = 'USUARIO_OSCAR'
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,owner,view_name,text
0,USUARIO_OSCAR,DEP_10_30,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,30)"
1,USUARIO_OSCAR,DEP_10_20,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,20)"


In [8]:
# Hacer una JOIN de las dos vistas anteriores.
QUERY = """
    SELECT A.EMP_NO, A.APELLIDO, A.OFICIO, A.DIR, A.FECHA_ALTA, A.SALARIO, A.COMISION, A.DEPT_NO
    FROM DEP_10_20 A
    JOIN DEP_10_30 B
    ON A.DEPT_NO = B.DEPT_NO
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,emp_no,apellido,oficio,dir,fecha_alta,salario,comision,dept_no
0,7782,Cerezo,Director,7839.0,1981-06-09,318500,None,10
1,7839,Rey,Presidente,NaN,1981-11-17,650000,None,10
2,7934,Muñoz,Empleado,7782.0,1982-01-23,169000,None,10
3,7782,Cerezo,Director,7839.0,1981-06-09,318500,None,10
4,7839,Rey,Presidente,NaN,1981-11-17,650000,None,10
5,7934,Muñoz,Empleado,7782.0,1982-01-23,169000,None,10
6,7782,Cerezo,Director,7839.0,1981-06-09,318500,None,10
7,7839,Rey,Presidente,NaN,1981-11-17,650000,None,10
8,7934,Muñoz,Empleado,7782.0,1982-01-23,169000,None,10


In [9]:
# PRUEBO CON OTRO METODO PARA RECUPERAR REGISTROS UNICOS.
QUERY = """
    SELECT * FROM DEP_10_20
    INTERSECT
    SELECT * FROM DEP_10_30
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,emp_no,apellido,oficio,dir,fecha_alta,salario,comision,dept_no
0,7782,Cerezo,Director,7839.0,1981-06-09,318500,None,10
1,7839,Rey,Presidente,NaN,1981-11-17,650000,None,10
2,7934,Muñoz,Empleado,7782.0,1982-01-23,169000,None,10


##### 72. Hacer una JOIN de la tabla DEPARTAMENTO y la vista de los departamentos 10 y 20.

In [10]:
QUERY = """
    SELECT A.EMP_NO, A.APELLIDO, A.OFICIO, A.DIR, A.FECHA_ALTA, A.SALARIO, NVL(A.COMISION,0) AS COMISION, A.DEPT_NO, 
    D.DNOMBRE, D.LOC
    FROM DEP_10_20 A
    INNER JOIN DEPARTAMENTO D
    ON A.DEPT_NO = D.DEPT_NO
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,emp_no,apellido,oficio,dir,fecha_alta,salario,comision,dept_no,dnombre,loc
0,7369,Sanchez,Empleado,7902.0,1980-12-17,104000,0,20,INVESTIGACIÓN,BILBAO
1,7566,Jimenez,Director,7839.0,1981-04-02,386750,0,20,INVESTIGACIÓN,BILBAO
2,7782,Cerezo,Director,7839.0,1981-06-09,318500,0,10,CONTABILIDAD,MADRID
3,7788,Gil,Analista,7566.0,1982-12-09,390000,0,20,INVESTIGACIÓN,BILBAO
4,7839,Rey,Presidente,NaN,1981-11-17,650000,0,10,CONTABILIDAD,MADRID
5,7876,Alonso,Empleado,7788.0,1983-01-12,143000,0,20,INVESTIGACIÓN,BILBAO
6,7902,Fernandez,Analista,7566.0,1981-12-03,390000,0,20,INVESTIGACIÓN,BILBAO
7,7934,Muñoz,Empleado,7782.0,1982-01-23,169000,0,10,CONTABILIDAD,MADRID


##### 73. Se va a realizar un programa de consulta de la información sobre enfermos. Los datos a mostrar serán sus apellidos, dirección, fecha de nacimiento y hospital en el que se encuentran. ¿Qué vista se definirá?. ¿Es posible modificar datos a través de la vista anterior?.

In [11]:
# Crear una vista desde los registros de la Tabla ENFERMOS con los datos
with engine.begin() as connection:
    QUERY = """
    CREATE VIEW REG_ENFERMO AS    
        SELECT E.APELLIDO, E.DIRECCION, E.FECHA_NAC, H.NOMBRE
        FROM ENFERMO E
        INNER JOIN OCUPACION O ON E.INSCRIPCION = O.INSCRIPCION
        INNER JOIN HOSPITAL H ON O.HOSPITAL_COD = H.HOSPITAL_COD
    """
    result = connection.execute(text(QUERY))

In [12]:
# verificar la creacion de la vista recuperando todas las creadas en el esquema
QUERY = """
    SELECT owner, view_name, text
    FROM ALL_VIEWS
    WHERE owner = 'USUARIO_OSCAR'
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,owner,view_name,text
0,USUARIO_OSCAR,REG_ENFERMO,"SELECT E.APELLIDO, E.DIRECCION, E.FECHA_NAC, H.NOMBRE\n FROM ENFERMO E\n INNER JOIN OCUPACION O ON E.INSCRIPCION = O.INSCRIPCION\n INNER JOIN HOSPITAL H ON O.HOSPITAL_COD = H.HOSPITAL_COD"
1,USUARIO_OSCAR,DEP_10_30,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,30)"
2,USUARIO_OSCAR,DEP_10_20,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,20)"


In [13]:
# recuperamos los datos de la vista
QUERY = """
    SELECT * FROM REG_ENFERMO
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,apellido,direccion,fecha_nac,nombre
0,Laguia M.,Recoletos 50,1967-06-23,Provincial
1,Serrano V.,Alcala 12,1960-05-21,Provincial
2,Fernandez M,Recoletos 50,1967-06-23,Provincial
3,Domin S.,Mayor 71,1942-01-01,General
4,Neal R.,Orense 11,1940-06-18,General
5,Cervantes M.,Peron 38,1952-02-29,La Paz
6,Miller G.,Lopez de Hoyos 2,1945-09-16,La Paz
7,Ruiz P.,Esquerdo 103,1980-12-26,La Paz
8,Fraser A.,Soto 3,1980-07-10,La Paz


##### Esta vista no se puede modificar pues es la combinacion de mas de dos tablas

##### 74. Estudiar esta vista.

CREATE VIEW EMP_CUA AS
SELECT DEPT_NO, SUM(SALARIO) SALARIOTOTAL
FROM EMPLEADO

In [14]:
try:
    QUERY = """
    CREATE VIEW EMP_CUA AS
    SELECT DEPT_NO, SUM(SALARIO) SALARIOTOTAL
    FROM EMPLEADO
    """
    df = pd.read_sql_query(QUERY, engine)
    df.columns = df.columns.str.upper()
    df
except Exception as e:
    print("Hubo un error:\n\n", e, '\n')

Hubo un error:

 (oracledb.exceptions.DatabaseError) ORA-00937: la función de grupo no es de grupo único
Help: https://docs.oracle.com/error-help/db/ora-00937/
[SQL: 
    CREATE VIEW EMP_CUA AS
    SELECT DEPT_NO, SUM(SALARIO) SALARIOTOTAL
    FROM EMPLEADO
    ]
(Background on this error at: https://sqlalche.me/e/20/4xp6) 



##### 75. Crear una vista para el departamento 10 con la cláusula with check option. ¿Qué ocurre?.

In [15]:
# Crear una vista desde los registros de la Tabla EMPLEADOS con los datos
with engine.begin() as connection:
    QUERY = """
    CREATE VIEW EMP_10 AS 
    SELECT * FROM EMPLEADO
    WHERE DEPT_NO = 10
    WITH CHECK OPTION
    """
    result = connection.execute(text(QUERY))

In [16]:
# verificar la creacion de la vista recuperando todas las creadas en el esquema
QUERY = """
    SELECT owner, view_name, text
    FROM ALL_VIEWS
    WHERE owner = 'USUARIO_OSCAR'
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,owner,view_name,text
0,USUARIO_OSCAR,EMP_10,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO"" FROM EMPLEADO\n WHERE DEPT_NO = 10\n WITH CHECK OPTION"
1,USUARIO_OSCAR,REG_ENFERMO,"SELECT E.APELLIDO, E.DIRECCION, E.FECHA_NAC, H.NOMBRE\n FROM ENFERMO E\n INNER JOIN OCUPACION O ON E.INSCRIPCION = O.INSCRIPCION\n INNER JOIN HOSPITAL H ON O.HOSPITAL_COD = H.HOSPITAL_COD"
2,USUARIO_OSCAR,DEP_10_30,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,30)"
3,USUARIO_OSCAR,DEP_10_20,"SELECT ""EMP_NO"",""APELLIDO"",""OFICIO"",""DIR"",""FECHA_ALTA"",""SALARIO"",""COMISION"",""DEPT_NO""\n FROM EMPLEADO\n WHERE DEPT_NO IN (10,20)"


In [17]:
# recuperamos los datos de la vista
QUERY = """
    SELECT * FROM EMP_10
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
df = pd.read_sql_query(QUERY, engine)
df

,emp_no,apellido,oficio,dir,fecha_alta,salario,comision,dept_no
0,7782,Cerezo,Director,7839.0,1981-06-09,318500,None,10
1,7839,Rey,Presidente,NaN,1981-11-17,650000,None,10
2,7934,Muñoz,Empleado,7782.0,1982-01-23,169000,None,10


##### La cláusula WITH CHECK OPTION en Oracle asegura que todas las operaciones de UPDATE y INSERT en la vista cumplen con la condición definida en la vista. Si se intenta una operación que no cumple con la condición de la vista, Oracle devuelve un error.

In [18]:
# cerrar la conexion
engine.dispose()

# Autor
### Oscar Gutierrez Leal
21 - 04 - 2024